In [20]:
import nest_asyncio
import asyncio
import pandas as pd
from telethon import TelegramClient

api_id = '28830994'
api_hash = '23259ea74c78c28822d2caf34af53c31'
phone_number = '+251935444997'

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Create the client with a different session file and increased timeout
client = TelegramClient('EthioMart_session', api_id, api_hash, connection_retries=5, timeout=10)

# Connect to Telegram
async def connect_to_telegram():
    await client.start(phone=phone_number)
    print("Connected to Telegram!")

# Fetch messages from a channel
async def fetch_messages(channel):
    messages = []
    async for message in client.iter_messages(channel, limit=10):
        messages.append({
            'sender': message.sender_id,
            'date': message.date,
            'content': message.text,
            'media': message.media
        })
    return messages


# Preprocess text
def preprocess_text(text):
    # Add your preprocessing steps here
    return text

# Run the connection
async def main():
    async with client:
        await connect_to_telegram()
        channel = "https://t.me/ZemenExpress"  # Replace with actual channel username
        messages = await fetch_messages(channel)
        
        # Apply preprocessing to the messages
        for msg in messages:
            if msg['content']:
                msg['content'] = preprocess_text(msg['content'])
        
        # Convert messages to DataFrame
        df = pd.DataFrame(messages)
        
        # Save to CSV
        df.to_csv('..\data\preprocessed_messages.csv', index=False, encoding='utf-8')
        print("Messages saved to preprocessed_messages.csv")

if __name__ == "__main__":
    asyncio.run(main())

INFO:telethon.network.mtprotosender:Connecting to 149.154.167.91:443/TcpFull...
DEBUG:telethon.network.mtprotosender:Connection attempt 1...
DEBUG:telethon.network.mtprotosender:Connection success!
DEBUG:telethon.network.mtprotosender:Starting send loop
DEBUG:telethon.network.mtprotosender:Starting receive loop
INFO:telethon.network.mtprotosender:Connection to 149.154.167.91:443/TcpFull complete!
DEBUG:telethon.network.mtprotosender:Waiting for messages to send...
DEBUG:telethon.extensions.messagepacker:Assigned msg_id = 7460966549955322092 to InvokeWithLayerRequest (17ad8253e90)
DEBUG:telethon.network.mtprotosender:Encrypting 1 message(s) in 76 bytes for sending
DEBUG:telethon.network.mtprotosender:Encrypted messages put in a queue to be sent
DEBUG:telethon.network.mtprotosender:Waiting for messages to send...
DEBUG:telethon.network.mtprotosender:Receiving items from the network...
DEBUG:telethon.network.mtprotosender:Handling bad salt for message 7460966549955322092
DEBUG:telethon.ne

Connected to Telegram!


DEBUG:telethon.network.mtprotosender:Handling RPC result for message 7460966556994735188
DEBUG:telethon.network.mtprotosender:Receiving items from the network...
DEBUG:telethon.extensions.messagepacker:Assigned msg_id = 7460966558369941780 to GetHistoryRequest (17ad8253d50)
DEBUG:telethon.network.mtprotosender:Encrypting 1 message(s) in 72 bytes for sending
DEBUG:telethon.network.mtprotosender:Encrypted messages put in a queue to be sent
DEBUG:telethon.network.mtprotosender:Waiting for messages to send...
DEBUG:telethon.extensions.messagepacker:Assigned msg_id = 7460966558397879668 to MsgsAck (17ace9d7cd0)
DEBUG:telethon.network.mtprotosender:Encrypting 1 message(s) in 36 bytes for sending
DEBUG:telethon.network.mtprotosender:Encrypted messages put in a queue to be sent
DEBUG:telethon.network.mtprotosender:Waiting for messages to send...
DEBUG:telethon.network.mtprotosender:Handling RPC result for message 7460966558369941780
DEBUG:telethon.network.mtprotosender:Receiving items from the

Messages saved to preprocessed_messages.csv


In [23]:
import pandas as pd

# Load the uploaded CSV file to examine its content
file_path = '../data/preprocessed_messages.csv'
data = pd.read_csv(file_path)

# Display the first few rows to verify the structure and content
data.head()


,sender,date,content,media
0,-1001307493052,2025-01-17 18:02:23+00:00,💥💥...................................💥💥\n\n📌 H...,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
1,-1001307493052,2025-01-17 18:02:11+00:00,💥💥...................................💥💥\n\n📌 H...,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
2,-1001307493052,2025-01-17 18:02:01+00:00,💥💥...................................💥💥\n\n📌 H...,"MessageMediaDocument(nopremium=False, spoiler=..."
3,-1001307493052,2025-01-17 11:14:54+00:00,💥💥...................................💥💥\n\n🎯El...,"MessageMediaPhoto(spoiler=False, photo=Photo(i..."
4,-1001307493052,2025-01-17 11:14:44+00:00,💥💥...................................💥💥\n\n🎯El...,"MessageMediaDocument(nopremium=False, spoiler=..."


In [24]:
import pandas as pd
import re

# Load the preprocessed_messages.csv file
file_path = '../data/preprocessed_messages.csv'
df = pd.read_csv(file_path)

# Define preprocessing and extraction functions
def preprocess_text(text):
    """Remove emojis, symbols, and clean text."""
    # Remove emojis
    text = re.sub(r'[^\w\s\u1200-\u137F]', '', text)  # Keep only Amharic and basic text
    # Normalize repetitive characters
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # Replace repeated characters (e.g., 🔥🔥🔥)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_price(text):
    """Extract price information from the text."""
    match = re.search(r'(\d+[.,]?\d*\s*ብር)', text)
    return match.group(1) if match else None

def extract_location(text):
    """Extract location information from the text."""
    match = re.search(r'(ቦሌ|ፒያሳ|ጊዮርጊስ|አዲስ አበባ)', text)  # Add more locations as needed
    return match.group(1) if match else None

def extract_contact_info(text):
    """Extract contact phone numbers."""
    matches = re.findall(r'\b\d{9,10}\b', text)  # Ethiopian phone number format
    return ', '.join(matches)

def extract_telegram_links(text):
    """Extract Telegram links or usernames."""
    matches = re.findall(r'@[a-zA-Z0-9_]+|https?://[^\s]+', text)
    return ', '.join(matches)

# Apply preprocessing and extraction functions
df['content_cleaned'] = df['content'].apply(preprocess_text)
df['price'] = df['content_cleaned'].apply(extract_price)
df['location'] = df['content_cleaned'].apply(extract_location)
df['contact_info'] = df['content_cleaned'].apply(extract_contact_info)
df['telegram_links'] = df['content_cleaned'].apply(extract_telegram_links)

# Save the structured data to a new CSV file
structured_file_path = '../data/structured_messages.csv'
df.to_csv(structured_file_path, index=False, encoding='utf-8')
print(f"Structured data saved to {structured_file_path}")


TypeError: expected string or bytes-like object, got 'float'